In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import torch.nn.functional as F
os.chdir("../")
from src.utilis.request_data import getDataTF, getDataHF
from src.utilis.models import decoding_pred
from src.data.processing_data import Format
from src.models.pipelines import Pipeline
from src.models.encoders import BERTencoder

2023-03-11 00:52:11.622736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 00:52:12.339891: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 00:52:12.339956: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 00:52:12.339963: W 

In [2]:
df = pd.read_csv(f"./inputs_data/data_dyda_da_test.csv", encoding="utf-8",  sep="|")

In [3]:
df.head()

,Utterance,Dialogue_ID,Label
0,"hey man , you wanna buy some weed ?",1,1
1,some what ?,1,3
2,"weed ! you know ? pot , ganja , mary jane some...",1,1
3,"oh , umm , no thanks .",1,0
4,i also have blow if you prefer to do a few lin...,1,1


In [4]:
# 
df.value_counts("Label")

Label
2    3534
3    2210
1    1278
0     718
dtype: int64

In [5]:
# number of distinct label i the dataset
len(df["Label"].unique())

4

In [6]:
# number of distinct dialogue i the dataset
len(df["Dialogue_ID"].unique())

1000

In [7]:
# number of utterance per dialogue
# df[["Dialogue_ID", "Label"]].groupby("Dialogue_ID").count().reset_index().sort_values("Label")

In [8]:
dimDialogAct, Contexts, Labels = Format("dyda_da", 5, "stacked").get_contexts_labels()

2023-03-11 00:52:19.381830: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 00:52:20.114059: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13581 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5


In [9]:
len(Contexts[2])

741

In [10]:
Labels[2]

<tf.Tensor: shape=(741, 20), dtype=int32, numpy=
array([[0, 1, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 1],
       ...,
       [0, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 1, 0]], dtype=int32)>

## Testing Pipeline working

In [11]:
perf, labelsHat = Pipeline("dyda_da", "stacked", 5, 'bert-base-cased', 6, "MLP").summary_exec()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.L

Epoch 1/500


2023-03-11 01:01:01.575279: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x561da41682b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-11 01:01:01.575342: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-03-11 01:01:01.587065: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-11 01:01:01.733879: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


259/259 [==============================] - 5s 8ms/step - loss: 0.4921 - val_loss: 0.4847
Epoch 2/500
259/259 [==============================] - 2s 7ms/step - loss: 0.4572 - val_loss: 0.4690
Epoch 3/500
259/259 [==============================] - 2s 7ms/step - loss: 0.4398 - val_loss: 0.4642
Epoch 4/500
259/259 [==============================] - 2s 7ms/step - loss: 0.4325 - val_loss: 0.4606
Epoch 5/500
259/259 [==============================] - 2s 7ms/step - loss: 0.4275 - val_loss: 0.4493
Epoch 6/500
259/259 [==============================] - 2s 7ms/step - loss: 0.4247 - val_loss: 0.4511
Epoch 7/500
259/259 [==============================] - 2s 7ms/step - loss: 0.4204 - val_loss: 0.4465
Epoch 8/500
259/259 [==============================] - 2s 8ms/step - loss: 0.4174 - val_loss: 0.4482
Epoch 9/500
259/259 [==============================] - 2s 7ms/step - loss: 0.4144 - val_loss: 0.4456
Epoch 10/500
259/259 [==============================] - 2s 7ms/step - loss: 0.4092 - val_loss: 0.4383
E

In [12]:
perf

,dataset_name,encoder_model,decoder_model,performance
0,dyda_da,bert-base-cased,MLP,0.591903


In [13]:
i = 200
yhat = decoding_pred(labelsHat.reshape(741, 5, 4))
y = np.array(Labels[2]).reshape(741, 5, 4)

In [14]:
yhat[i]

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [15]:
y[i]

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0]], dtype=int32)